In [ ]:
import pandas as pd
import warnings

In [ ]:
# cargar el archivo CSV expdata.csv
df_l1= pd.read_csv('NE_L1.csv', delimiter=';')
df_l2= pd.read_csv('NE_L2.csv', delimiter=';')
df_l3= pd.read_csv('NE_L3.csv', delimiter=';')
df_l4= pd.read_csv('NE_L4.csv', delimiter=';')
df_l5= pd.read_csv('NE_L5.csv', delimiter=';')
df_l6= pd.read_csv('NE_L6.csv', delimiter=';')
df_l7= pd.read_csv('NE_L6.csv', delimiter=';')
df_null= pd.read_csv('DataNull.csv', delimiter=';')
warnings.filterwarnings('ignore', category=FutureWarning, message='Setting an item of incompatible dtype')


In [ ]:
df_l1_grouped = df_l1.groupby(['U_Estilo', 'U_Silueta', 'U_Team', 'U_Descrip_Color', 'U_Segmento','U_Liga', 'U_Coleccion_NE', 'U_Genero', 'U_Descripcion', 'U_Temporalidad']).size().reset_index(name='count')
df_l1_grouped = df_l1_grouped.drop(columns=['count'])
df_l2_grouped = df_l2.groupby(['U_Estilo', 'U_Silueta', 'U_Team', 'U_Descrip_Color', 'U_Segmento','U_Liga', 'U_Coleccion_NE', 'U_Genero', 'U_Descripcion', 'U_Temporalidad']).size().reset_index(name='count')
df_l2_grouped = df_l2_grouped.drop(columns=['count'])
df_l3_grouped = df_l3.groupby(['U_Estilo', 'U_Silueta', 'U_Team', 'U_Descrip_Color', 'U_Segmento','U_Liga', 'U_Coleccion_NE', 'U_Genero', 'U_Descripcion', 'U_Temporalidad']).size().reset_index(name='count')
df_l3_grouped = df_l3_grouped.drop(columns=['count'])
df_l4_grouped = df_l4.groupby(['U_Estilo', 'U_Silueta', 'U_Team', 'U_Descrip_Color', 'U_Segmento','U_Liga', 'U_Coleccion_NE', 'U_Genero', 'U_Descripcion', 'U_Temporalidad']).size().reset_index(name='count')
df_l4_grouped = df_l4_grouped.drop(columns=['count'])
df_l5_grouped = df_l5.groupby(['U_Estilo', 'U_Silueta', 'U_Team', 'U_Descrip_Color', 'U_Segmento','U_Liga', 'U_Coleccion_NE', 'U_Genero', 'U_Descripcion', 'U_Temporalidad']).size().reset_index(name='count')
df_l5_grouped = df_l5_grouped.drop(columns=['count'])
df_l6_grouped = df_l6.groupby(['U_Estilo', 'U_Silueta', 'U_Team', 'U_Descrip_Color', 'U_Segmento','U_Liga', 'U_Coleccion_NE', 'U_Genero', 'U_Descripcion', 'U_Temporalidad']).size().reset_index(name='count')
df_l6_grouped = df_l6_grouped.drop(columns=['count'])
df_l7_grouped = df_l7.groupby(['U_Estilo', 'U_Silueta', 'U_Team', 'U_Descrip_Color', 'U_Segmento','U_Liga', 'U_Coleccion_NE', 'U_Genero', 'U_Descripcion', 'U_Temporalidad']).size().reset_index(name='count')
df_l7_grouped = df_l7_grouped.drop(columns=['count'])

In [ ]:
# 1. Convertir 'U_Estilo' a string en todos los DataFrames (evita errores en el merge)

df_l1_grouped['U_Estilo'] = df_l1_grouped['U_Estilo'].astype(str)
df_l2_grouped['U_Estilo'] = df_l2_grouped['U_Estilo'].astype(str)
df_l3_grouped['U_Estilo'] = df_l3_grouped['U_Estilo'].astype(str)
df_l4_grouped['U_Estilo'] = df_l4_grouped['U_Estilo'].astype(str)
df_l5_grouped['U_Estilo'] = df_l5_grouped['U_Estilo'].astype(str)
df_l6_grouped['U_Estilo'] = df_l5_grouped['U_Estilo'].astype(str)
df_l7_grouped['U_Estilo'] = df_l5_grouped['U_Estilo'].astype(str)

# 2. Definir las columnas que queremos completar
columnas_comunes = [
    'U_Silueta', 'U_Team', 'U_Descrip_Color', 'U_Segmento',
    'U_Liga', 'U_Coleccion_NE', 'U_Genero', 'U_Descripcion','U_Temporalidad'
]

# 3. Orden de prioridad para completar datos (primero df_l1, luego df_l2, luego df_l3)
dataframes_a_combinar = [df_l2_grouped, df_l3_grouped, df_l4_grouped, df_l5_grouped,df_l6_grouped,df_l7_grouped]

# 4. Inicializar el DataFrame final con los datos originales
df_final = df_l1_grouped.copy()

# 5. Proceso de combinación en cascada
for df_temp in dataframes_a_combinar:
    # Merge con el DataFrame actual (left join para mantener todo de df_final)
    df_merged = df_final.merge(
        df_temp,
        on='U_Estilo',
        how='left',
        suffixes=('', '_temp')
    )
    
    # Completar valores faltantes en las columnas comunes
    for col in columnas_comunes:
        if col in df_final.columns and f'{col}_temp' in df_merged.columns:
            # Prioriza los valores existentes y rellena con los del nuevo DataFrame
            df_merged[col] = df_merged[col].fillna(df_merged[f'{col}_temp'])
            df_merged.drop(f'{col}_temp', axis=1, inplace=True)
    
    # Añadir registros únicos del DataFrame temporal que no están en df_final
    df_final = pd.concat([
        df_merged,
        df_temp[~df_temp['U_Estilo'].isin(df_final['U_Estilo'])]
    ], ignore_index=True)

# 6. Eliminar posibles duplicados (por si acaso)
df_final = df_final.drop_duplicates(subset='U_Estilo', keep='first')

# 7. Resultado final
print(f"✔ DataFrame final generado con {df_final.shape[0]} filas y {df_final.shape[1]} columnas")
print(f"✔ Registros originales: {len(df_l1_grouped)} | Registros añadidos: {len(df_final) - len(df_l1_grouped)}")

# Opcional: Sobrescribir df si es necesario
df = df_final.copy()

In [ ]:

df_null['U_Estilo'] = df_null['ItemName'].str.split('/').str[0]
df_null['U_Descripcion'] = df_null['ItemName'].str.replace('\s+', ' ', regex=True).str.strip().str.split('/').str[1]
df_null['U_Descripcion'] = (
    df_null['ItemName']
    .str.replace('\s+', ' ', regex=True)  # Reemplaza múltiples espacios por uno solo
    .str.strip()  # Elimina espacios al inicio y final
    .str.split('/')  # Divide por '/'
    .str[1]  # Toma la segunda parte
    .str.strip()  # Elimina espacios al inicio y final del resultado
)
df_null['U_Talla'] = df_null['ItemName'].str.split('/').str[2]
# Columnas a completar (excluyendo ItemCode y Empresa)
columnas_completar = ['U_Estilo', 'U_Silueta', 'U_Team', 'U_Descrip_Color', 'U_Segmento',
                     'U_Liga', 'U_Coleccion_NE', 'U_Genero', 'U_Descripcion','U_Temporalidad']

# Columnas adicionales a mantener
columnas_extra = ['ItemCode', 'Empresa','ItemName','U_Talla']

# Asegurarse de trabajar con las columnas correctas
df = df[columnas_completar]
df_null = df_null[columnas_completar + columnas_extra]

# Convertir U_Estilo al mismo tipo de dato en ambos DataFrames
df['U_Estilo'] = df['U_Estilo'].astype(str)
df_null['U_Estilo'] = df_null['U_Estilo'].astype(str)

# Crear un diccionario de referencia desde df (sin duplicados por U_Estilo)
df_reference = df.drop_duplicates('U_Estilo').set_index('U_Estilo').reset_index()

# Versión optimizada con merge
temp_df = df_null[columnas_extra + ['U_Estilo']].merge(
    df_reference, 
    on='U_Estilo', 
    how='left',
    suffixes=('', '_ref')
)

# Completar valores nulos
for col in columnas_completar[1:]:  # Empezamos desde 1 para omitir U_Estilo
    mask = df_null[col].isnull()
    df_null.loc[mask, col] = temp_df.loc[mask, col]

In [ ]:
team_licenses = {
    'LOS ANGELES DODGERS': 'MLB',
    'NEW YORK YANKEES': 'MLB',
    'PITTSBURGH PIRATES': 'MLB',
    'SAN FRANCISCO GIANTS': 'MLB',
    'SEATTLE MARINERS': 'MLB',
    'TAMPA BAY RAYS': 'MLB',
    'NEW ERA BRANDED': 'NEW ERA BRANDED',
    'NO APLICA': 'NO APLICA',
    'NEW ENGLAND PATRIOTS': 'NFL',
    'HOUSTON TEXANS': 'NFL',
    'BALTIMORE RAVENS': 'NFL',
    'TORONTO BLUE JAYS': 'MLB',
    'HOUSTON ASTROS': 'MLB',
    'GREEN BAY PACKERS': 'NFL',
    'BOSTON RED SOX': 'MLB',
    'BALTIMORE ORIOLES': 'MLB',
    'ST. LOUIS CARDINALS': 'MLB',
    'SEATTLE SEAHAWKS': 'NFL',
    'DALLAS COWBOYS': 'NFL',
    'PITTSBURGH STEELERS': 'NFL',
    'MIAMI DOLPHINS': 'NFL',
    'STARWARS': 'ENTERTAINMENT',
    'DALLAS MAVERICKS': 'NBA',
    'LOS ANGELES LAKERS': 'NBA',
    'NEW ORLEANS SAINTS': 'NFL',
    'JACKSONVILLE JAGUARS': 'NFL',
    'CLEVELAND BROWNS': 'NFL',
    'NEW YORK KNICKS': 'NBA',
    'SAN ANTONIO SPURS': 'NBA',
    'WASHINGTON NATIONALS': 'MLB',
    'OAKLAND ATHLETICS': 'MLB',
    'DETROIT TIGERS': 'MLB',
    'ANAHEIM ANGELS': 'MLB',
    'NASCAR': 'MOTORSPORT',
    'NEW YORK METS': 'MLB',
    'PHILADELPHIA PHILLIES': 'MLB',
    'CHICAGO WHITE SOX': 'MLB',
    'SAN DIEGO PADRES': 'MLB',
    'CLEVELAND INDIANS': 'MLB',
    'DENVER BRONCOS': 'NFL',
    'BUFFALO BILLS': 'NFL',
    'ATLANTA FALCONS': 'NFL',
    'CHICAGO BEARS': 'NFL',
    'BROOKLYN NETS': 'NBA',
    'CHICAGO BULLS': 'NBA',
    'SAN FRANCISCO 49ERS': 'NFL',
    'INDIANAPOLIS COLTS': 'NFL',
    'ARIZONA CARDINALS': 'NFL',
    'OAKLAND RAIDERS': 'NFL',
    'LOS ANGELES RAMS': 'NFL',
    'TAMPA BAY BUCCANEERS': 'NFL',
    'GOLDEN STATE WARRIORS': 'NBA',
    'BOSTON CELTICS': 'NBA',
    'CHICAGO CUBS': 'MLB',
    'TWEETY PIE': 'ENTERTAINMENT',
    'TOY STORY': 'ENTERTAINMENT',
    'ATLANTA BRAVES': 'MLB',
    'SPIDERMAN': 'ENTERTAINMENT',
    'DEADPOOL': 'ENTERTAINMENT',
    'PHILADELPHIA EAGLES': 'NFL',
    'DETROIT LIONS': 'NFL',
    'MILWAUKEE BUCKS': 'NBA',
    'MIAMI HEAT': 'NBA',
    'FLORIDA MARLINS': 'MLB',
    'MCLAREN RACING': 'MOTORSPORT',
    'MICKEY MOUSE': 'ENTERTAINMENT',
    'WASHINGTON REDSKINS': 'NFL',
    'LAS VEGAS RAIDERS': 'NFL',
    'NEW YORK GIANTS': 'NFL',
    'CINCINNATI REDS': 'MLB',
    'HULK': 'ENTERTAINMENT',
    'KANSAS CITY CHIEFS': 'NFL',
    'NEW ORLEANS PELICANS': 'NBA',
    'DENVER NUGGETS': 'NBA',
    'MINNESOTA VIKINGS': 'NFL',
    'MIAMI MARLINS': 'MLB',
    'TENNESSEE TITANS': 'NFL',
    'UNKNOWN': 'MLB',
    'NO APLICA': 'NBA',
    'CAPTAIN AMERICA': 'ENTERTAINMENT',
    'PHOENIX SUNS': 'NBA',
    'MLB GENERIC LOGO': 'MLB',
    'BROOKLYN DODGERS': 'MLB',
    'SYLVESTER': 'ENTERTAINMENT',
    'ORLANDO MAGIC': 'NBA',
    'ARIZONA DIAMONDBACKS': 'MLB',
    'TEXAS RANGERS': 'MLB',
    'LOONEY TUNES': 'ENTERTAINMENT',
    'INDUSTRIA INC': 'ENTERTAINMENT',
    'FORMULA E': 'MOTORSPORT',
    'CLUB DEPORTIVO OLIMPIA': 'HONDURAS SOCCER',
    'CAROLINA PANTHERS': 'NFL',
    'MINNESOTA TWINS': 'MLB',
    'JUSTICE LEAGUE': 'ENTERTAINMENT',
    'COLORADO ROCKIES': 'MLB',
    'TORONTO RAPTORS': 'NBA',
    'ANIMANIACS': 'ENTERTAINMENT',
    'MONSTERS INC': 'ENTERTAINMENT',
    'HOUSTON OILERS': 'NFL',
    'SUPERMAN': 'ENTERTAINMENT',
    'BATMAN': 'ENTERTAINMENT',
    'KANSAS CITY ROYALS': 'MLB',
    'LOS ANGELES CLIPPERS': 'NBA',
    'MINNIE MOUSE': 'ENTERTAINMENT',
    'CLUB SOCIAL DEPORTIVO MUNICIPAL': 'GUATEMALA SOCCER LEAGUE',
    'EMF': 'ENTERTAINMENT',
    'UTAH JAZZ': 'NBA',
    'NEW YORK JETS': 'NFL',
    'HOUSTON ROCKETS': 'NBA',
    'LED ZEPPELIN': 'ENTERTAINMENT',
    'GUATEMALA': 'MARCA PAIS',
    'NONE': 'NEW ERA BRANDED',
    'LEHIGH VALLEY IRON PIGS': 'MILB',
    'FRESNO GRIZZLIES': 'MILB',
    'NFL GENERIC LOGO': 'NFL',
    'PLUTO': 'ENTERTAINMENT',
    'MONTREAL EXPOS': 'MLB',
    'MILWAUKEE BREWERS': 'MLB',
    'GENERIC DISNEY': 'ENTERTAINMENT',
    'BUGS BUNNY': 'ENTERTAINMENT',
    'MEMPHIS CHICKS': 'MILB',
    'PIGLET': 'ENTERTAINMENT',
    'LOS ANGELES CHARGERS': 'NFL',
    'GOOFY': 'ENTERTAINMENT',
    'TOM & JERRY': 'ENTERTAINMENT',
    'HARTFORD YARD GOATS': 'MILB',
    'PHILADELPHIA 76ERS': 'NBA',
    'CINCINNATI BENGALS': 'NFL',
    'ATLANTA HAWKS': 'NBA',
    'FEAR OF GOD': 'FEAR OF GOD',
    'NBA LOGO': 'NBA',
    'LA CASA DE PAPEL': 'ENTERTAINMENT',
    'NFL OFFICIAL LOGO': 'NFL',
    'TELLAECHE': 'TELLAECHE',
    'COMUNICACIONES FC': 'GUATEMALA SOCCER LEAGUE',
    'LOS ANGELES ANGELS': 'MLB',
    'PANAMA': 'MARCA PAIS',
    'TEAM GLISTEN': 'ENTERTAINMENT',
    'TIGGER': 'ENTERTAINMENT',
    'PUERTO RICO': 'FEDERACION DE BASEBALL PUERTO RICO',
    'EL SALVADOR': 'MARCA PAIS',
    'GOLDEN STATE WARRIOR': 'NBA',
    'CHARLOTTE HORNETS': 'NBA',
    'NFL GENERIC SUPERBOW': 'NFL',
    'BIZARRAP': 'ENTERTAINMENT',
    'NONE': 'NONE LICENSED',
    'NFL ALL OVER': 'NFL',
    'PUERTO RICO': 'MARCA PAIS',
    'DOMINICAN REPUBLIC': 'FEDERACION DOMINICANA DE BASEBALL',
    'KOUMORI': 'NEW ERA BRANDED',
    'WINNIE THE POOH': 'ENTERTAINMENT',
    'OKLAHOMA CITY THUNDER': 'NBA',
    'MINNESOTA TIMBERWOLVES': 'NBA',
    'MLB PROPERTIES ALL OVER': 'MLB',
    'NBA ALL OVER': 'NBA',
    'CLEVELAND CAVALIERS': 'NBA',
    'WASHINGTON': 'NFL',
    'DISNEY': 'ENTERTAINMENT',
    'TAZ': 'ENTERTAINMENT',
    'DAFFY DUCK': 'ENTERTAINMENT',
    'MLB ALL OVER': 'MLB',
    'PORTLAND TRAIL BLAZE': 'NBA',
    'SCOOBY DOO': 'ENTERTAINMENT',
    'BROOKLYN CYCLONES': 'MILB',
    'SACRAMENTO KINGS': 'NBA',
    'DOMINICAN REPUBLIC C': 'BASEBALL FEDERATION',
    'TEEN TITAN STAR FIRE': 'ENTERTAINMENT',
    'COLUMBUS CREW': 'MLS',
    'SEATTLE SOUNDERS': 'MLS',
    'MCLAREN F1 RACING': 'MOTORSPORT',
    'MLB LOGO': 'MLB',
    'NEW ERA BRANDED': 'MLB',
    'NONE': 'NON LICENSED',
    'SPACE JAM': 'NBA',
    'MANCHESTER UNITED FC': 'EUROPEAN SOCCER',
    'CHELSEA FC': 'EUROPEAN SOCCER',
    'DUCATI': 'MOTORSPORT',
    'MERCEDES BENZ': 'MOTORSPORT',
    'NBA ALL STAR GAME': 'NBA',
    'RENAULT F1 RACING': 'MOTORSPORT',
    'NEW YORK RED BULLS': 'MLS',
    'INTER MIAMI': 'MLS',
    'NEW YORK CITY FC': 'MLS',
    'NFL LOGO': 'NFL',
    'RENAULT F1': 'MOTORSPORT',
    'TEEN TITAN CYBORG': 'ENTERTAINMENT',
    'L.A. GALAXY': 'MLS',
    'TEEN TITAN RAVEN': 'ENTERTAINMENT',
    'LAKELAND TIGERS': 'MILB',
    'RED BULL F1': 'MOTORSPORT',
    'CALIFORNIA ANGELS': 'MLB',
    'READING FIGHTIN PHILS': 'MILB',
    'CLEVELAND 2022': 'MLB',
    'LOS ANGELES FC': 'MLS',
    'WASHINGTON COMMANDER': 'NFL',
    'DETROIT PISTONS': 'NBA',
    'CLEVELAND GUARDIANS': 'MLB',
    'CHELSEA FC LION CREST': 'EUROPEAN SOCCER',
    'TEEN TITAN ROBIN': 'ENTERTAINMENT',
    'WBC MEXICO': 'WBC',
    'WBC VENEZUELA': 'WBC',
    'WBC PUERTO RICO': 'WBC',
    'MLB ALL STAR GAME LOGO': 'MLB',
    'MLB GENERIC WORLD SERIES': 'MLB',
    'TUCSON SIDEWINDERS': 'MILB',
    'WB 100TH LOONEY TUNES MASHUPS': 'ENTERTAINMENT',
    'MEMPHIS GRIZZLIES': 'NBA',
    'OKLAHOMA CITY THUNDE': 'NBA',
    'PORTLAND TRAIL BLAZERS': 'NBA',
    'HAAS FORMULA 1': 'MOTORSPORT',
    'RED BULL F1 RACING': 'MOTORSPORT',
    'WB HARRY POTTER DEAT': 'ENTERTAINMENT',
    'WB HARRY POTTER DEATHLY HOLLOW PT 2': 'ENTERTAINMENT',
    'WBC PANAMA': 'WBC',
    'WILE E COYOTE': 'ENTERTAINMENT',
    'WB HARRY POTTER DEATHLY HOLLOW PT 2': 'WARNER BROS',
    'GRAPEFRUIT LEAGUE LO': 'MLB',
    'LAS VEGAS AVIATORS': 'MILB',
    'MCLAREN AUTOMOTIVE': 'MOTORSPORT',
    'BOSTON BRAVES': 'MLB',
    'PHILADELPHIA PHILLIE': 'MLB',
    'CLUB DEPORTIVO OLIMP': 'HONDURAS SOCCER',
    'CANGREJEROS DE SANTU': 'PUERTO RICO BASEBALL',
    'CANGREJEROS DE SANTU': 'LIGA DE BEISBOL PROFESIONAL ROBERTO CLEMENTE',
    'BEAVIS AND BUTT-HEAD': 'ENTERTAINMENT',
    'HONDURAS': 'MARCA PAIS',
    'VISA CASH APP RACING': 'MOTORSPORT',
    'WILLY WONKA': 'ENTERTAINMENT',
    'ALPINE RACING': 'MOTORSPORT',
    'BUGS BUNNY': 'WARNER BROS',
    'TRUE': 'NEW ERA BRANDED',
    'BRAND NEW ERA': 'NEW ERA BRANDED',
    'MANDALORIAN': 'ENTERTAINMENT',
    'IRONMAN': 'ENTERTAINMENT',
    'INCREDIBLE HULK': 'ENTERTAINMENT',
    'NFL OILERS': 'NFL',
    'UNKNOWN': 'ENTERTAINMENT',
    'WBC DOMINICAN REPUBLIC': 'WBC',
    'MLB MULTI TEAM': 'MLB',
    'A NIGHTMARE ON ELM S': 'ENTERTAINMENT',
    'WILE E COYOTE': 'WARNER BROS',
    'INTERNATIONAL SPEEDW': 'MOTORSPORT',
    'SPONGEBOB': 'ENTERTAINMENT',
    'HEY ARNOLD': 'ENTERTAINMENT',
    'BEAVIS AND BUTT-HEAD': 'NEW ERA BRANDED',
    'FRIDAY THE 13TH': 'ENTERTAINMENT',
    'DONALD DUCK': 'ENTERTAINMENT',
    'OAKLAND ATHLETICS': 'MARCA PAIS',
    'ENTREGAS DESCONOCIDAS': 'NO APLICA',
    'PATRICK STAR': 'ENTERTAINMENT',
    'ELMER FUDD': 'ENTERTAINMENT',
    'DOWN EAST WOOD DUCKS': 'ENTERTAINMENT',
    'ELMER FUDD': 'WARNER BROS',
    'NFL OILERS': 'MARCA PAIS',
    'CHICAGO BEARS': 'MARCA PAIS',
    'CORPUS CRISTI HOOKS': 'MILB',
    'ENTREGAS A PERSONAL': 'NO APLICA'
}

In [ ]:
 #1. Identificar los registros donde U_Liga está en blanco
registros_en_blanco = df_null[df_null['U_Liga'].isna() | (df_null['U_Liga'] == '')]

# 2. Para estos registros, buscar el valor de U_Team en el diccionario team_licenses
# y asignarlo a U_Liga
for index, row in registros_en_blanco.iterrows():
    equipo = row['U_Team']
    if equipo in team_licenses:
        df_null.at[index, 'U_Liga'] = team_licenses[equipo]

# Opcional: Verificar cuántos registros se actualizaron
actualizados = df_null[df_null['U_Liga'].notna() & (df_null['U_Liga'] != '')]
print(f"Se actualizaron {len(actualizados)} registros")


# Guardar el resultado si es necesario
df_null.to_csv('DATA_LIMPIA.csv', sep=';', index=False)
df_null.head()